In [5]:
!pip install seaborn

In [6]:
import os
import subprocess
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from struct import unpack

# First, change to the parent directory if we're in the exe directory
current_dir = os.getcwd()
if current_dir.endswith('exe'):
    os.chdir('..')
    current_dir = os.getcwd()

def get_bmp_bit_depth(file_path):
    try:
        with open(file_path, 'rb') as f:
            # Read BMP header
            f.seek(0)
            header = f.read(54)
            # Check if it's a BMP file
            if header[0:2] != b'BM':
                return None
            # Get bit depth (offset 28)
            bit_depth = unpack('H', header[28:30])[0]
            return bit_depth
    except Exception as e:
        print(f"Error reading BMP header for {file_path}: {e}")
        return None

def process_image(input_file, method, exe_name):
    try:
        # Get absolute paths
        current_dir = os.getcwd()
        input_path = os.path.abspath(os.path.join(current_dir, "input", input_file))
        output_dir = os.path.abspath(os.path.join(current_dir, "output"))
        exe_dir = os.path.abspath(os.path.join(current_dir, "exe"))
        
        # Check if input file exists and is 24-bit BMP
        if not os.path.exists(input_path):
            print(f"Input file not found: {input_path}")
            return None
            
        bit_depth = get_bmp_bit_depth(input_path)
        if bit_depth != 24:
            print(f"Skipping {input_file} (not a 24-bit BMP, found {bit_depth}-bit)")
            return None
            
        # Get the base name of the input file
        name_without_ext = os.path.splitext(input_file)[0]
        
        # Create output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)
        
        # Change to exe directory
        os.chdir(exe_dir)
        
        # Construct the command with absolute paths
        cmd = [
            f"./{exe_name}",
            input_path,
            os.path.join(output_dir, f"Result{name_without_ext}.bmp"),
            method
        ]
        
        print(f"Processing {input_file} ({bit_depth}-bit) using {method} method...")
        print(f"Command: {' '.join(cmd)}")
        
        # Run the command and capture output
        result = subprocess.run(cmd, 
                              check=True, 
                              capture_output=True, 
                              text=True,
                              encoding='utf-8')
        
        # Print the full output for debugging
        print("Output:", result.stdout)
        print("OhYESDADDY:", result.stderr)
        
        # Change back to original directory
        os.chdir(current_dir)
        
        # Extract execution time from output
        if result.stdout:
            for line in result.stdout.split('\n'):
                if "Processing completed in" in line:
                    try:
                        # Extract time value
                        time_str = line.split("Processing completed in")[1].split("ms")[0].strip()
                        time_ms = int(time_str)
                        print(f"Execution time: {time_ms} ms")
                        return time_ms
                    except Exception as e:
                        print(f"Error parsing time from output: {e}")
                        print(f"Line was: {line}")
        
        print("Warning: Could not find execution time in output")
        return None
                
    except subprocess.CalledProcessError as e:
        print(f"Error processing {input_file} with {method}: {e}")
        print(f"Command output: {e.output}")
        print(f"Command stderr: {e.stderr}")
        if os.getcwd() != current_dir:
            os.chdir(current_dir)
        return None
    except Exception as e:
        print(f"Error processing {input_file} with {method}: {e}")
        if os.getcwd() != current_dir:
            os.chdir(current_dir)
        return None

def main():
    # Print current working directory
    print(f"Current working directory: {os.getcwd()}")
    
    # Dictionary mapping methods to their executable names with correct method parameters
    method_exes = {
        "sequential": "sequential.exe",
        "omp": "omp_version.exe",
        "mpi": "mpi_version.exe",
        "cuda": "cuda_version.exe"
    }
    
    # Method display names for graphs and statistics
    method_display_names = {
        "sequential": "Sequential",
        "omp": "OpenMP",
        "mpi": "MPI",
        "cuda": "CUDA"
    }
    
    # Verify directories exist
    input_dir = os.path.abspath("input")
    if not os.path.exists(input_dir):
        print(f"Error: Input directory not found at {input_dir}")
        return
        
    # Get list of BMP files and check bit depth
    input_files = []
    print("\nChecking BMP files:")
    for f in os.listdir(input_dir):
        if f.lower().endswith('.bmp'):
            file_path = os.path.join(input_dir, f)
            bit_depth = get_bmp_bit_depth(file_path)
            print(f"{f}: {bit_depth}-bit")
            if bit_depth == 24:
                input_files.append(f)
    
    print(f"\nFound {len(input_files)} compatible 24-bit BMP files out of {len([f for f in os.listdir(input_dir) if f.lower().endswith('.bmp')])} total BMP files")
    
    # Create DataFrame to store results
    results = []
    
    # Process each compatible file with each method
    for input_file in input_files:
        input_path = os.path.join(input_dir, input_file)
        file_size = os.path.getsize(input_path) / (1024 * 1024)  # Size in MB
        
        for method, exe_name in method_exes.items():
            execution_time = process_image(input_file, method, exe_name)
            if execution_time is not None:
                results.append({
                    'File': input_file,
                    'Method': method_display_names[method],
                    'Time (ms)': execution_time,
                    'Size (MB)': file_size
                })
    
    if not results:
        print("No results to analyze!")
        return
        
    # Create DataFrame
    df = pd.DataFrame(results)
    
    # Create plots
    plt.style.use('seaborn-v0_8')
    
    # 1. Box plot
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='Method', y='Time (ms)', data=df)
    plt.title('Execution Time Distribution by Method')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('boxplot_comparison.png')
    plt.close()
    
    # 2. Violin plot
    plt.figure(figsize=(12, 6))
    sns.violinplot(x='Method', y='Time (ms)', data=df)
    plt.title('Execution Time Distribution (Violin Plot)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('violin_comparison.png')
    plt.close()
    
    # 3. Scatter plot
    plt.figure(figsize=(12, 6))
    for method in method_display_names.values():
        method_data = df[df['Method'] == method]
        plt.scatter(method_data['Size (MB)'], method_data['Time (ms)'], label=method, alpha=0.6)
    plt.xlabel('File Size (MB)')
    plt.ylabel('Execution Time (ms)')
    plt.title('Execution Time vs File Size')
    plt.legend()
    plt.tight_layout()
    plt.savefig('size_vs_time.png')
    plt.close()
    
    # 4. Bar plot
    plt.figure(figsize=(12, 6))
    avg_times = df.groupby('Method')['Time (ms)'].mean()
    avg_times.plot(kind='bar')
    plt.title('Average Execution Time by Method')
    plt.ylabel('Average Time (ms)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('average_times.png')
    plt.close()
    
    # Print statistics with more detail
    print("\nPerformance Statistics:")
    print("=" * 50)
    
    # Group by Method and calculate statistics
    stats = df.groupby('Method').agg({
        'Time (ms)': ['count', 'mean', 'median', 'std', 'min', 'max']
    }).round(2)
    
    print("\nDetailed Statistics:")
    print(stats)
    
    # Calculate and print speedup for each file size category
    print("\nSpeedup Analysis:")
    print("=" * 50)
    
    # Create file size categories
    df['Size Category'] = pd.cut(df['Size (MB)'], 
                                bins=[0, 1, 5, 10, float('inf')],
                                labels=['Small (<1MB)', 'Medium (1-5MB)', 'Large (5-10MB)', 'Very Large (>10MB)'])
    
    # Calculate speedup by size category
    for size_cat in df['Size Category'].unique():
        size_data = df[df['Size Category'] == size_cat]
        if len(size_data) > 0:
            print(f"\nSize Category: {size_cat}")
            seq_mean = size_data[size_data['Method'] == 'Sequential']['Time (ms)'].mean()
            for method in method_display_names.values():
                if method != 'Sequential':
                    method_mean = size_data[size_data['Method'] == method]['Time (ms)'].mean()
                    if method_mean > 0:
                        speedup = seq_mean / method_mean
                        print(f"{method}: {speedup:.2f}x speedup")
    
    # Save detailed results
    df.to_csv('performance_results.csv', index=False)
    stats.to_csv('performance_statistics.csv')
    
    print("\nResults and statistics saved to:")
    print("- performance_results.csv")
    print("- performance_statistics.csv")
    print("- boxplot_comparison.png")
    print("- violin_comparison.png")
    print("- size_vs_time.png")
    print("- average_times.png")

if __name__ == "__main__":
    main()

Current working directory: c:\Users\user\Desktop\FInal DSPC\New folder

Checking BMP files:
1.bmp: 24-bit
10.bmp: 32-bit
11.bmp: 24-bit
12.bmp: 24-bit
13.bmp: 24-bit
14.bmp: 24-bit
15.bmp: 24-bit
16.bmp: 24-bit
17.bmp: 24-bit
18.bmp: 32-bit
19.bmp: 24-bit
2.bmp: 24-bit
20.bmp: 24-bit
21.bmp: 24-bit
22.bmp: 24-bit
23.bmp: 24-bit
24.bmp: 24-bit
25.bmp: 24-bit
26.bmp: 24-bit
27.bmp: 24-bit
28.bmp: 24-bit
29.bmp: 24-bit
3.bmp: 24-bit
30.bmp: 24-bit
31.bmp: 24-bit
32.bmp: 24-bit
33.bmp: 24-bit
34.bmp: 24-bit
35.bmp: 24-bit
36.bmp: 24-bit
37.bmp: 24-bit
38.bmp: 24-bit
39.bmp: 24-bit
4.bmp: 24-bit
40.bmp: 24-bit
41.bmp: None-bit
42.bmp: 24-bit
43.bmp: 24-bit
44.bmp: 24-bit
45.bmp: 24-bit
46.bmp: 24-bit
47.bmp: 24-bit
48.bmp: 24-bit
49.bmp: 24-bit
5.bmp: 24-bit
50.bmp: 24-bit
6.bmp: 24-bit
7.bmp: 32-bit
8.bmp: 24-bit
9.bmp: 24-bit
sample1.bmp: 24-bit

Found 47 compatible 24-bit BMP files out of 51 total BMP files
Processing 1.bmp (24-bit) using sequential method...
Command: ./sequential.exe c: